In [ ]:
import matplotlib.pyplot as plt
import polars as pl

from monkey import get_df, find_latest_csv, to_int, to_struct


# pathname = 'C:/Users/maple/Downloads/results*.csv'
pathname = '/home/chrlz/Downloads/results*.csv'

csv = find_latest_csv(pathname)
print(csv)


pl.Config.set_tbl_hide_column_data_types(True)


df = get_df(csv).with_columns(
    pl.col('charStats').str.split(',').list.eval(
        pl.element().cast(pl.UInt16)).list.to_struct(),
    to_struct('_id').name.keep(),
    # pl.col('_id').alias('original_id'),
)

def print_stats(df: pl.DataFrame):
    mean = df['wpm'].mean()
    restarts = df['restartCount'].sum()/len(df)
    print(f'Mean: {mean}')
    print(f'Restarts: {restarts}')
    print('Worst:')
    display(df.sort('wpm').head(15).sort(
        'index', descending=True).select('wpm', 'index').transpose())
    print('Earliest:')
    display(df.head(15).select('wpm', 'index').transpose())
    print('Best:')
    display(df.sort('wpm', descending=True).head(20))

# plt.hist(df.select('wpm'), bins=40)
# print_stats(df)
# for d in df:
#     print(d.name, len(d.unique()))


pl.Config.set_tbl_hide_column_data_types(False)
print(len(open(csv, 'rb').read()))
print(df.estimated_size())
pl.Config.set_fmt_str_lengths(10000)
df.with_columns(to_int('_id').alias('reconstructed')).filter(pl.col('isPb'))

print_stats(df)